In [1]:
import urllib.request
import os
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords



import re 
import nltk 

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from nltk import sent_tokenize, word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn import svm
import xgboost as xgb


import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
train = pd.read_excel("/content/edited.xlsx")
train.head()


,1,tweet,label
0,2,#studiolife #aislife #requires #passion #dedic...,0
1,3,@user #white #supremacists want everyone to se...,1
2,4,safe ways to heal your #acne!! #altwaystohe...,0
3,5,is the hp and the cursed child book up for res...,0
4,6,"3rd #bihday to my amazing, hilarious #nephew...",0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
train['tweet'] = train['tweet'].astype(str)

In [16]:
def remove_pattern(input_txt,pattern):
  r= re.findall(pattern,input_txt)
  for i in r:
    input_txt = re.sub(i,'',input_txt)
  return input_txt

In [17]:
train['tidy_tweet'] = np.vectorize(remove_pattern)(train['tweet'],"@[\w]*")

train['tidy_tweet'] = train['tidy_tweet'].str.replace("[^a-zA-Z#]"," ")
train['tidy_tweet']= train['tidy_tweet'].apply(lambda x : ' '.join([w for w in x.split() if len(w)>3])) 

In [18]:
tokenizer=RegexpTokenizer(r'\w+')
en_stopwords=set(stopwords.words("english"))
nltk.download('wordnet')
lemmatizer = nltk.stem.WordNetLemmatizer()
def getLemmatizedTweets(tweet):
    tweet= tweet.lower()
    tweet = remove_pattern(tweet,"@[\w]*")
    tweet = tweet.replace("[^a-zA-Z#]"," ")

    #Tokenize
    tokens=tokenizer.tokenize(tweet)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    lem_tokens = [lemmatizer.lemmatize(token) for token in new_tokens]
    clean_tweet =' '.join(lem_tokens)
    return clean_tweet



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
train.head()


,1,tweet,label,tidy_tweet
0,2,#studiolife #aislife #requires #passion #dedic...,0,#studiolife #aislife #requires #passion #dedic...
1,3,@user #white #supremacists want everyone to se...,1,#white #supremacists want everyone #birds #mov...
2,4,safe ways to heal your #acne!! #altwaystohe...,0,safe ways heal your #acne #altwaystoheal #heal...
3,5,is the hp and the cursed child book up for res...,0,cursed child book reservations already where w...
4,6,"3rd #bihday to my amazing, hilarious #nephew...",0,#bihday amazing hilarious #nephew ahmir uncle ...


In [20]:
clean_tweet = train['tweet'].apply(getLemmatizedTweets)
clean_tweet_df = pd.DataFrame(clean_tweet)
train['tidy_tweet'] = clean_tweet_df

In [21]:
tfidf_vectorizer = TfidfVectorizer(max_df = 0.90,min_df= 2,max_features=1000)  
#Convert a collection of raw documents to a matrix of TF-IDF features.

tfidf = tfidf_vectorizer.fit_transform(train['tidy_tweet'])
#Learn vocabulary and idf, return document-term matrix.

In [22]:
xtrain_tfidf,xvalid_tfidf,ytrain,yvalid = train_test_split(tfidf,train['label'])
xtrain_tfidf = tfidf[ytrain.index]
xvalid_index = tfidf[yvalid.index]

In [23]:
svc = svm.SVC(kernel='linear',C=1,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
f1_score(yvalid,prediction_int)


0.8411214953271028

In [24]:

svc = svm.SVC(kernel='linear',C=1,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
print(f1_score(yvalid,prediction_int))
print(accuracy_score(yvalid,prediction_int))
print(precision_score(yvalid,prediction_int))

0.8400374181478018


NameError: ignored

In [26]:
svc = svm.SVC(kernel='polynomial',C=1,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
print(f1_score(yvalid,prediction_int))
print(accuracy_score(yvalid,prediction_int))
print(precision_score(yvalid,prediction_int))

ValueError: ignored

In [ ]:
svc = svm.SVC(kernel='rbf',C=1,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
print(f1_score(yvalid,prediction_int))
print(accuracy_score(yvalid,prediction_int))
print(precision_score(yvalid,prediction_int))

In [ ]:
svc = svm.SVC(kernel='sigmoid',C=1,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
print(f1_score(yvalid,prediction_int))
print(accuracy_score(yvalid,prediction_int))
print(precision_score(yvalid,prediction_int))

In [ ]:
svc = svm.SVC(kernel='rbf',C=2,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
print(f1_score(yvalid,prediction_int))
print(accuracy_score(yvalid,prediction_int))
print(precision_score(yvalid,prediction_int))

In [ ]:
svc = svm.SVC(kernel='rbf',C=4,probability= True).fit(xtrain_tfidf,ytrain)
prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1]>= 0.3
prediction_int = prediction_int.astype(np.int)
print(f1_score(yvalid,prediction_int))
print(accuracy_score(yvalid,prediction_int))
print(precision_score(yvalid,prediction_int))